In [2]:
import numpy as np
from scipy.stats import bernoulli, norm, poisson
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from pgmpy.factors.discrete import DiscreteFactor

/home/warrior/miniconda3/envs/inference/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### DiscreteFactors

In [4]:
distX = DiscreteFactor(
    variables=['X'],
    cardinality=[3],
    values=[0.45,0.30,0.25],
    state_names={'X':['1','2','3']}
)
print(distX)

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


In [6]:
distX.sample(5)

,X
0,1
1,1
2,1
3,2
4,2


In [7]:
joint = DiscreteFactor(
    variables=['X', 'Y'],
    cardinality=[3, 2],
    values=[0.25,0.20, 0.20,0.10, 0.15,0.10], #given in tuples (x1,y1, x2,y2, x3,y3)
    state_names={
        'X':['1','2','3'],
        'Y':['1','2']}
)
print(joint)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(1) |     0.2500 |
+------+------+------------+
| X(1) | Y(2) |     0.2000 |
+------+------+------------+
| X(2) | Y(1) |     0.2000 |
+------+------+------------+
| X(2) | Y(2) |     0.1000 |
+------+------+------------+
| X(3) | Y(1) |     0.1500 |
+------+------+------------+
| X(3) | Y(2) |     0.1000 |
+------+------+------------+


In [10]:
joint.sample(3)

,X,Y
0,3,1
1,1,2
2,2,1


In [9]:
print(joint.marginalize(variables=['Y'], inplace=False))

+------+----------+
| X    |   phi(X) |
+======+==========+
| X(1) |   0.4500 |
+------+----------+
| X(2) |   0.3000 |
+------+----------+
| X(3) |   0.2500 |
+------+----------+


In [33]:
print(joint.marginalize(variables=['X'], inplace=False))

+------+----------+
| Y    |   phi(Y) |
+======+==========+
| Y(1) |   0.6000 |
+------+----------+
| Y(2) |   0.4000 |
+------+----------+


In [11]:
# P(Y|X)
print(joint / distX)

+------+------+------------+
| X    | Y    |   phi(X,Y) |
+======+======+============+
| X(1) | Y(1) |     0.5556 |
+------+------+------------+
| X(1) | Y(2) |     0.4444 |
+------+------+------------+
| X(2) | Y(1) |     0.6667 |
+------+------+------------+
| X(2) | Y(2) |     0.3333 |
+------+------+------------+
| X(3) | Y(1) |     0.6000 |
+------+------+------------+
| X(3) | Y(2) |     0.4000 |
+------+------+------------+


### Sampling from Distributions

In [1]:
from pgmpy.factors.discrete.CPD import TabularCPD
from pgmpy.models import BayesianNetwork
from pgmpy.sampling import BayesianModelSampling

/home/warrior/miniconda3/envs/inference/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Sampling model
<center>$z \sim p(Z)$<center>

<center>$x \sim P(X|Z=z)$<center>

<center>$y \sim P(Y|X=x)$<center>

In [2]:
pZ = TabularCPD(
    variable='Z',
    variable_card=2,
    values=[[.65],[.35]],
    state_names={'Z':['0','1']}
)

In [3]:
pXgZ = TabularCPD(
    variable='X',
    variable_card=2,
    values=[[.8, .6],
            [.2, .4]],
    evidence=['Z'],
    evidence_card=[2],
    state_names={'X':['0','1'],
                 'Z':['0','1']}
)

In [4]:
pYgX = TabularCPD(
    variable='Y',
    variable_card=3,
    values=[[.1, .8],
            [.2, .1],
            [.7, .1]
            ],
    evidence=['X'],
    evidence_card=[2],
    state_names={'Y':['1','2', '3'],
                 'X':['0','1']}
)

In [10]:
model = BayesianNetwork([('Z','X'),('X','Y')]) # Graph: Z->X->Y
model.add_cpds(pZ, pXgZ, pYgX) # Conditional Probability Distributions in the model

In [8]:
generator = BayesianModelSampling(model) # Sampling object
generator.forward_sample(size=5)

Generating for node: Y: 100%|██████████| 3/3 [00:00<00:00, 824.95it/s]


,Z,X,Y
0,0,0,3
1,1,0,3
2,0,1,1
3,0,0,3
4,0,1,1


In [9]:
samples = generator.forward_sample(size=100)
samples.shape
samples.head()

Generating for node: Y: 100%|██████████| 3/3 [00:00<00:00, 780.38it/s]


,Z,X,Y
0,0,0,3
1,1,0,3
2,0,0,3
3,1,0,3
4,1,0,2


### Expected value of Y $E[Y]$

In [32]:
samples['Y'].apply(int).mean()

2.16